In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn


In [26]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
# Convert y to int (0/1) instead of bool to avoid ONNX conversion issues
y = data['checked'].astype(int)
X = data.drop(['Ja', 'Nee', 'checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [12]:
# ============================================================================
# BAD MODEL: RandomForest without any preprocessing
# ============================================================================
print("=" * 70)
print("Training BAD MODEL (RandomForest - No Preprocessing)")
print("=" * 70)

# Introduce bias: flip 50% of y=1, persoon_geslacht_vrouw=0 to persoon_geslacht_vrouw=1
# and flip 50% of y=0, persoon_geslacht_vrouw=1 to persoon_geslacht_vrouw=0
X_train_biased = X_train.copy()
if 'persoon_geslacht_vrouw' in X_train_biased.columns:
    np.random.seed(42)  # For reproducibility
    
    # Case 1: Flip 50% of y=1, persoon_geslacht_vrouw=0 to persoon_geslacht_vrouw=1
    mask1 = (y_train == 1) & (X_train_biased['persoon_geslacht_vrouw'] == 0)
    indices_to_flip_1 = X_train_biased[mask1].index
    
    n_to_flip_1 = int(len(indices_to_flip_1) * 0.5)
    indices_to_flip_selected_1 = np.random.choice(indices_to_flip_1, size=n_to_flip_1, replace=False)
    X_train_biased.loc[indices_to_flip_selected_1, 'persoon_geslacht_vrouw'] = 1
    
    # Case 2: Flip 50% of y=0, persoon_geslacht_vrouw=1 to persoon_geslacht_vrouw=0
    mask2 = (y_train == 0) & (X_train_biased['persoon_geslacht_vrouw'] == 1)
    indices_to_flip_2 = X_train_biased[mask2].index
    
    n_to_flip_2 = int(len(indices_to_flip_2) * 0.5)
    indices_to_flip_selected_2 = np.random.choice(indices_to_flip_2, size=n_to_flip_2, replace=False)
    X_train_biased.loc[indices_to_flip_selected_2, 'persoon_geslacht_vrouw'] = 0
    
    print(f"Introduced bias:")
    print(f"  - Flipped {len(indices_to_flip_selected_1)} gender labels (50% of y=1, gender=0 cases to gender=1)")
    print(f"  - Flipped {len(indices_to_flip_selected_2)} gender labels (50% of y=0, gender=1 cases to gender=0)")
else:
    print("Warning: 'persoon_geslacht_vrouw' column not found in training data")

bad_classifier = RandomForestClassifier(random_state=0)
bad_pipeline = Pipeline(steps=[('classification', bad_classifier)])

# Train the bad model on biased data
bad_pipeline.fit(X_train_biased, y_train)

# Evaluate the bad model
y_pred_bad = bad_pipeline.predict(X_test)
bad_accuracy = accuracy_score(y_test, y_pred_bad)
print(f'\nOverall Accuracy of BAD model: {bad_accuracy:.4f}')

# Note: Partition testing (sensitive feature analysis) has been moved to partitioning_tests.ipynb


Training BAD MODEL (RandomForest - No Preprocessing)
Introduced bias:
  - Flipped 3782 gender labels (50% of y=1, gender=0 cases to gender=1)
  - Flipped 20090 gender labels (50% of y=0, gender=1 cases to gender=0)

Overall Accuracy of BAD model: 0.8766


In [8]:
# ============================================================================
# GOOD MODEL: AdaBoost with DecisionTree base estimator
# ============================================================================
print("=" * 70)
print("Training GOOD MODEL (AdaBoost with DecisionTree base estimator)")
print("=" * 70)

# Create AdaBoostClassifier with DecisionTreeClassifier(max_depth=2) as base estimator
good_classifier = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=2),
    n_estimators=100,
    learning_rate=1.0,
    random_state=0
)
good_pipeline = Pipeline(steps=[('classification', good_classifier)])

# Train the good model
good_pipeline.fit(X_train, y_train)

# Evaluate the good model
y_pred_good = good_pipeline.predict(X_test)
good_accuracy = accuracy_score(y_test, y_pred_good)
print(f'\nOverall Accuracy of GOOD model: {good_accuracy:.4f}')

# Note: Partition testing (sensitive feature analysis) has been moved to partitioning_tests.ipynb

Training GOOD MODEL (AdaBoost with DecisionTree base estimator)

Overall Accuracy of GOOD model: 0.9460


In [ ]:
# ============================================================================
# Convert BAD model to ONNX
# ============================================================================
print("\n" + "=" * 70)
print("Converting BAD model to ONNX")
print("=" * 70)

bad_onnx_model = convert_sklearn(
    bad_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Check accuracy of converted bad model
bad_sess = rt.InferenceSession(bad_onnx_model.SerializeToString())
y_pred_bad_onnx = bad_sess.run(None, {'X': X_test.values.astype(np.float32)})
bad_accuracy_onnx = accuracy_score(y_test, y_pred_bad_onnx[0])
print(f'Accuracy of BAD ONNX model: {bad_accuracy_onnx:.4f}')

# Save bad model
onnx.save(bad_onnx_model, "model/model_2_2.onnx")
print("Saved BAD model to: model/model_2_2.onnx")



Converting BAD model to ONNX
Accuracy of BAD ONNX model: 0.8766
Saved BAD model to: model/model_2.onnx


In [ ]:
# ============================================================================
# Convert GOOD model to ONNX
# ============================================================================
print("\n" + "=" * 70)
print("Converting GOOD model to ONNX")
print("=" * 70)

# Use the correct number of features (after removing sensitive features if bias was detected)
good_onnx_model = convert_sklearn(
    good_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Check accuracy of converted good model
good_sess = rt.InferenceSession(good_onnx_model.SerializeToString())
y_pred_good_onnx = good_sess.run(None, {'X': X_test.values.astype(np.float32)})
good_accuracy_onnx = accuracy_score(y_test, y_pred_good_onnx[0])
print(f'Accuracy of GOOD ONNX model: {good_accuracy_onnx:.4f}')

# Save good model
onnx.save(good_onnx_model, "model/model_2_1.onnx")
print("Saved GOOD model to: model/model_2_1.onnx")



Converting GOOD model to ONNX
Accuracy of GOOD ONNX model: 0.9460
Saved GOOD model to: model/model_1.onnx


In [ ]:
# ============================================================================
# Summary Comparison
# ============================================================================
print("\n" + "=" * 70)
print("MODEL COMPARISON SUMMARY")
print("=" * 70)
print(f"\nBAD Model (RandomForest, no preprocessing):")
print(f"  Overall Accuracy: {bad_accuracy:.4f}")

print(f"\nGOOD Model (AdaBoost, post-processing with threshold per group):")
print(f"  Overall Accuracy: {good_accuracy:.4f}")

print(f"\nDifference: {good_accuracy - bad_accuracy:+.4f}")

# Verify saved models can be loaded
print("\n" + "=" * 70)
print("Verifying saved models...")
print("=" * 70)

bad_loaded = rt.InferenceSession("model/model_2_2.onnx")
good_loaded = rt.InferenceSession("model/model_2_1.onnx")

y_pred_bad_loaded = bad_loaded.run(None, {'X': X_test.values.astype(np.float32)})
y_pred_good_loaded = good_loaded.run(None, {'X': X_test.values.astype(np.float32)})

bad_loaded_acc = accuracy_score(y_test, y_pred_bad_loaded[0])
good_loaded_acc = accuracy_score(y_test, y_pred_good_loaded[0])

print(f"BAD model loaded accuracy: {bad_loaded_acc:.4f}")
print(f"GOOD model loaded accuracy: {good_loaded_acc:.4f}")
print("\n✓ Both models saved and verified successfully!")



MODEL COMPARISON SUMMARY

BAD Model (RandomForest, no preprocessing):
  Overall Accuracy: 0.8772

GOOD Model (AdaBoost, post-processing with threshold per group):
  Overall Accuracy: 0.9460

Difference: +0.0687

Verifying saved models...


NameError: name 'X_test_fair' is not defined